In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [2]:
train.head()

,id,path,label
0,TRAIN_0000,./train/TRAIN_0000.wav,1
1,TRAIN_0001,./train/TRAIN_0001.wav,2
2,TRAIN_0002,./train/TRAIN_0002.wav,4
3,TRAIN_0003,./train/TRAIN_0003.wav,5
4,TRAIN_0004,./train/TRAIN_0004.wav,4


In [67]:
import numpy as np
import librosa
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout,Reshape
from tqdm import tqdm
from keras.callbacks import EarlyStopping

In [68]:
preprocess_data = []
preprocess_labels = []
for i, row in train.iterrows():
    # Load the audio file using librosa
    y, sr = librosa.load(row['path'], sr=44100)
    
    # Extract Mel spectrogram features using librosa
    mel_spec = librosa.feature.melspectrogram(y=y, sr=44100)
    
    # Resize the spectrogram to a fixed size (e.g., 128x128) for consistency
    resized_spectrogram = np.resize(mel_spec, (128, 128))
    
    # Calculate the minimum and maximum values of the resized spectrogram
    min_val = np.min(resized_spectrogram)
    max_val = np.max(resized_spectrogram)
    
    # Check if the minimum and maximum values are the same
    if min_val == max_val:
        normalized_spectrogram = np.zeros_like(resized_spectrogram)  # Set all values to 0
    else:
        # Perform normalization
        normalized_spectrogram = (resized_spectrogram - min_val) / (max_val - min_val)
    
    # Append the preprocessed data and label to the respective lists
    preprocess_data.append(normalized_spectrogram)
    preprocess_labels.append(row['label'])

In [69]:
# Convert the preprocessed data and labels to NumPy arrays
X = np.array(preprocess_data)
y = np.array(preprocess_labels)

# Step 2: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Reshape the data for the CRNN model
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], X_val.shape[2], 1))


In [70]:
# Step 4: Build the CRNN model with LSTM layer
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Reshape((-1, 64)))  # Reshape for LSTM input
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 6 classes in total

In [38]:
'''
# Step 4: Build the CRNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 6 classes in total
'''

In [72]:
#overfitting prevention techniques
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with callbacks
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/10
125/125 [==============================] - 2197s 18s/step - loss: 1.7906 - accuracy: 0.1700 - val_loss: 1.7950 - val_accuracy: 0.1578 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 4654s 37s/step - loss: 1.7901 - accuracy: 0.1790 - val_loss: 1.7932 - val_accuracy: 0.1578 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 2346s 19s/step - loss: 1.7899 - accuracy: 0.1750 - val_loss: 1.7951 - val_accuracy: 0.1748 - lr: 0.0010
Epoch 4/10
101/125 [=======================>......] - ETA: 8:04 - loss: 1.7890 - accuracy: 0.1791

KeyboardInterrupt: 

In [71]:
'''
# Step 5: Compile and train the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))
'''

Epoch 1/10
  6/125 [>.............................] - ETA: 27:04 - loss: 1.7915 - accuracy: 0.2083

KeyboardInterrupt: 

In [40]:
# Step 6: Evaluate the model
score = model.evaluate(X_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Validation loss: 3.0619616508483887
Validation accuracy: 0.30069929361343384


In [ ]:
# Step 4: Build the CRNN model with LSTM layer
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Reshape((-1, 64)))  # Reshape for LSTM input
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 6 classes in total

In [66]:
# Step 4: Build the CRNN model with LSTM layer
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))  # Additional Conv2D layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Reshape((-1, 32)))  # Reshape for LSTM input
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 6 classes in total

# Step 5: Compile and train the model with overfitting prevention techniques
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with callbacks
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

Epoch 1/10
125/125 [==============================] - 1600s 13s/step - loss: 1.7910 - accuracy: 0.1695 - val_loss: 1.7938 - val_accuracy: 0.1568 - lr: 0.0010
Epoch 2/10
 24/125 [====>.........................] - ETA: 19:21 - loss: 1.7890 - accuracy: 0.1602

KeyboardInterrupt: 

In [41]:
preprocess_test = []

for i, row in test.iterrows():
    # Load the audio file using librosa
    y, sr = librosa.load(row['path'], sr=44100)
    
    # Extract Mel spectrogram features using librosa
    mel_spec = librosa.feature.melspectrogram(y=y, sr=44100)
    
    # Resize the spectrogram to a fixed size (e.g., 128x128) for consistency
    resized_spectrogram = np.resize(mel_spec, (128, 128))
    
    # Calculate the minimum and maximum values of the resized spectrogram
    min_val = np.min(resized_spectrogram)
    max_val = np.max(resized_spectrogram)
    
    # Check if the minimum and maximum values are the same
    if min_val == max_val:
        normalized_spectrogram = np.zeros_like(resized_spectrogram)  # Set all values to 0
    else:
        # Perform normalization
        normalized_spectrogram = (resized_spectrogram - min_val) / (max_val - min_val)
    
    # Append the preprocessed data and label to the respective lists
    preprocess_test.append(normalized_spectrogram)

In [42]:
# Convert the preprocessed data and labels to NumPy arrays
x_test = np.array(preprocess_test)

# Reshape the data for the CRNN model
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

In [43]:
pred = model.predict(x_test)

59/59 [==============================] - 39s 610ms/step


In [46]:
predicted_labels = np.argmax(pred, axis=1)

In [47]:
predicted_labels

array([0, 4, 2, ..., 5, 2, 4], dtype=int64)

In [48]:
sub['label'] = predicted_labels
sub.head()

,id,label
0,TEST_0000,0
1,TEST_0001,4
2,TEST_0002,2
3,TEST_0003,1
4,TEST_0004,5


In [49]:
sub.to_csv('sub_CRNN+mel_spec.csv', index = False)